In [1]:
#read params
# process data


In [51]:
import os
import yaml
import pandas as pd
import numpy as np
import argparse

class process_data:
    
    def get_data_func(self, data_path):
        df = pd.read_csv(data_path, sep=',')
        return df
    
    def split_raw_data(self, df, config_path):
        cols = [col.replace(' ', '_') for col in df.columns]
        print(cols)
        df.to_csv(config_path, index=False, header=cols, sep=',')

    def __init__(self, path=''):
        if(path == ''):
            self.path = 'params.yaml'
        self.raw_data = dict()
        with open('../' + self.path) as f:
            self.raw_data  = yaml.safe_load(f)
        print(self.raw_data)
        df = self.get_data_func(self.raw_data['data_source']['s3_source'])
        self.split_raw_data(df, self.raw_data['load_data']['raw_dataset_csv'])


In [52]:
process_data()

{'base': {'project': 'winequality-project', 'random_state': 42, 'target_col': 'TARGET'}, 'data_source': {'s3_source': '../data_given/winequality.csv'}, 'load_data': {'raw_dataset_csv': '../data/raw/winequality.csv'}, 'split_data': {'train_path': '../data/processed/train_winequality.csv', 'test_path': '../data/processed/test_winequality.csv', 'test_size': 0.2}, 'estimators': {'ElasticNet': {'params': {'alpha': 0.9, 'l1_ratio': 0.4}}}, 'model_dir': 'saved_models'}
['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'TARGET']


In [44]:
"a a".replace(" ", "_")

'a_a'